In [1]:
import numpy as np
import yaml

import light.utils.result as result
from light.utils import helper_functions as _helper_functions


/opt/homebrew/Caskroom/mambaforge/base/envs/light-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Make launch file

These cells define some default settings, for specific catalogs settings see below. 

In [2]:
# good starting point for the magnitude distribution
REFERENCE_DICT_MAGNITUDE_PRIOR_LIST = [
    {
        'mu': 101.924, 
        'sigma': 195.502, 
        'eps': -2.472, 
        'eps_2': 0.059, 
        'pre_a0': -0.251, 
        'pre_b0': 0.218, 
        'pre_a1': 0.012, 
        'pre_b1': 0.163, 
        'pre_a2': 0.312, 
        'pre_b2': 0.537, 
        'pre_a3': -0.002, 
        'pre_b3': 0.020, 
        'pre_a4': 0.186, 
        'pre_b4': 0.409, 
        'pre_a5': 0.049, 
        'pre_b5': 0.399, 
    },
]

def add_debug_kwargs(kwargs_sampler):
    kwargs_sampler['num_warmup'] = 1000
    kwargs_sampler['num_posterior_samples'] = 400
    kwargs_sampler['max_tree_depth'] = 10
    kwargs_sampler['target_accept_prob'] = 0.8

    kwargs_sampler['priors'] = dict(
        ng_bar_min=0.01 * q,
        ng_bar_max=0.2 * q,
        epsilon_g_min=0.1,
        epsilon_g_max=0.8,
        field_cut_exp_min=-0.9,
        field_cut_exp_max=0,
        gamma_min=0.3,
        gamma_max=2.3,
        A_s_min=40,
        A_s_max=8e3,
        n_s_min=-1.4,
        n_s_max=0.3,
        f_faint_max=0.5,
        f_faint_min=0.2,
        mu_faintest_value=-17
    )

In [3]:
def save_settings(
    launch_filename,
    kwargs_catalog,
    results_location,
    data_location,
    power_spectrum_model='analytical-phenom', 
    redshift_error=False,
    special_deep_pixels=0,
    debug=False, 
    redshift_error_model=None,
    folder_catalog='./mydata/',
):

    kwargs_sampler = \
    dict(
        num_svi_samples=0,
        num_posterior_samples_per_batch=200,
        num_warmup=2000,
        num_posterior_samples=3000,
        max_tree_depth=10,
        target_accept_prob=0.9,
        num_chains=4,
        gibbs_mode=0
    )

    kwargs_sampler['magnitude_model_params_init'] = REFERENCE_DICT_MAGNITUDE_PRIOR_LIST[0]

    if debug is False:
        if(redshift_error==False):
            
            kwargs_sampler['priors'] = dict(
                ng_bar_min=0.01 * q,
                ng_bar_max=0.2 * q,
                A_s_min=8e2,
                A_s_max=8e3,
                n_s_min=-0.8,
                n_s_max=0.3,
            )
        else:
            if(redshift_error_model==None):
                kwargs_sampler['priors'] = dict(
                    ng_bar_min=0.01 * q,
                    ng_bar_max=0.2 * q,
                    A_s_min=1e5,
                    A_s_max=1e9,
                    n_s_min=-2,
                    n_s_max=0.3,
                    
                )
            elif(redshift_error_model=='convolution-1d'):
                kwargs_sampler['priors'] = dict(
                    ng_bar_min=0.005 * q,
                    ng_bar_max=0.2 * q,
                    A_s_min=800,
                    A_s_max=8000,
                    n_s_min=-2,
                    n_s_max=0.3,
                )
    else:
        add_debug_kwargs(kwargs_sampler)

    dict_to_add = dict(
        epsilon_g_min=0.5,
        epsilon_g_max=1.4,
        field_cut_exp_min=-0.8,
        field_cut_exp_max=0.2,
        gamma_min=0.5,
        gamma_max=2.5,
        f_faint_max=0.5,
        f_faint_min=0.2,
        mu_faintest_value=-17
    )
    kwargs_sampler['priors'] = {**kwargs_sampler['priors'], **dict_to_add}
    
    kwargs_field={
            'power_spectrum_model':power_spectrum_model,
            'set_zero_mode_to_zero': True,
            'integration_method': 'stretch',
            'galaxy_model':'simple-bias-exp-gamma',
            'replace_FT_with_packing': True,
            'redshift_error_model': redshift_error_model
        }
    
    kwargs_cosmology = dict(
        cosmo_ref_parameters=dict(H0=73, Omega_M=0.25),
        cosmology_model='flatLCDM',
        cosmo_numerics = dict(z_min=1e-8, z_max=None, z_steps=100_000)
    )
    
    kwargs_magnitude_model = \
    dict(
        n_pre=6,
        n_post=0,
        magnitude_model_name='NF_sigmoid_regularized',
    )
    
    if(special_deep_pixels!=0):
        kwargs_catalog['special_deep_pixels'] = special_deep_pixels
    
    if(redshift_error):
        kwargs_catalog['redshift_error_settings'] = True

    name_catalog = _helper_functions.get_data_name(**kwargs_catalog)
    catalog_settings_path = folder_catalog + name_catalog + '_settings'
    
    dict1 = dict(
        kwargs_sampler=kwargs_sampler,
        kwargs_magnitude_model=kwargs_magnitude_model,
        catalog_settings_path=catalog_settings_path,
        kwargs_cosmology=kwargs_cosmology,
        kwargs_field=kwargs_field,
        results_location=results_location,
        data_location=data_location,
    )
    
    with open(launch_filename, 'w') as yaml_file:
        yaml.dump(dict1, yaml_file)

    print('Saved settings. ')

    args_post = result.post_processing_default_command_line_args(dict1)

    return args_post

The settings you are most likely to change are here: 

In [4]:
power_spectrum_model = ['analytical-phenom', 'cosmopower', 'flat'][0] # options for power spectrum
# make sure to choose compatible values from the catalog generated previously
Z_offset, q, n, box_size, m_threshold = 300, 0.01, 10, 150, 17
folder_catalog = './my_data/'
kwargs_catalog = dict(
    box_shape_d=(n,n,n),
    m_threshold=m_threshold,
    Z_offset=Z_offset,
    catalog_name='catalog'
)
data_location = './my_data/'
results_location = './where_you_want_your_results_stored/'

settings = save_settings(
    launch_filename='../launch.yaml',
    power_spectrum_model=power_spectrum_model,
    kwargs_catalog=kwargs_catalog,
    redshift_error=False,
    special_deep_pixels=0,
    folder_catalog='./my_data/',
    data_location=data_location,
    results_location=results_location,
)

Saved settings. 
./my_data/dim_3_mtresh_17_matrix_schechter_False_z_off_300_catalog_pixels_X_10_Y_10_Z_10_redshift_pixelation_settings
results_location
data_location


In [5]:
settings

{'kwargs_catalog': {'M_cut_high': inf,
  'Z_offset': 300,
  'additional_information': {'angular_diameter_distance': 278.9647627176866,
   'faintest_absolute_magnitude': -20.583105,
   'fraction_never_observed': 0.7936290460695324,
   'number_observed_galaxies': 1020.0},
  'angle_box_0': 0.5377023196001302,
  'angle_box_1': 0.5377023196001302,
  'box_shape_d': [10, 10, 10],
  'box_size_d': [150, 150, 150],
  'catalog_file_name': 'dim_3_mtresh_17_matrix_schechter_False_z_off_300_catalog_pixels_X_10_Y_10_Z_10_redshift_pixelation',
  'debug': False,
  'galaxies_percentage': 0.01,
  'geometry': 'toy',
  'm_threshold': 17,
  'pixel_length_in_Mpc_comoving': [15.0, 15.0, 15.0],
  'pixelation': 'redshift',
  'redshift_num_bins': 10,
  'sample_from_schechter': False,
  'special_deep_pixels': [],
  'zmax': 2},
 'kwargs_magnitude_model': {'n_pre': 6,
  'n_post': 0,
  'magnitude_model_name': 'NF_sigmoid_regularized'},
 'kwargs_sampler': {'num_svi_samples': 0,
  'num_posterior_samples_per_batch': 20

In [6]:
! mkdir where_you_want_your_results_stored

mkdir: where_you_want_your_results_stored: File exists
